# CoreMS Tutorial
`CoreMS` framework for mass spectrum

### 1. Create the reader object

In [ ]:
from pathlib import Path
from corems.mass_spectrum.input.massList import ReadMassList
from corems.transient.input.BrukerSolarix import ReadBrukerSolarix

file_location = Path.cwd() / "ESI_NEG_SRFA.d"

reader_obj = ReadBrukerSolarix(file_location)

bruker_transient = reader_obj.get_transient()

#reader_obj = ReadMassList(file_location, isCentroid=True)
#reader_obj = ReadMassList(file_location)
#reader_obj = ReadMassList(file_location)
#reader_obj = ReadHDF_BoosterMassSpectrum(file_location, polarity)
reader_obj.file_location

### 2. Create the mass spectrum object


In [ ]:
mass_spectrum = bruker_transient.get_mass_spectrum(plot_result=False, auto_process=True)
mass_spectrum.mz_exp, mass_spectrum.abundance


### 3. Access mass spectrum peak objects

In [ ]:
kb = {'C': 1, 'H': 2}

for mspeak in mass_spectrum.sort_by_abundance():
    
    print(mspeak.mz_exp, mspeak.abundance, mspeak.resolving_power, 
          mspeak.signal_to_noise, mspeak.kmd, mspeak.is_assigned)
    
    mspeak.change_kendrick_base(kb)



### 4. Mass spectrum calibration

In [ ]:
from corems.molecular_id.search.findOxigenPeaks import FindOxygenPeaks
from corems.mass_spectrum.calc.CalibrationCalc import FreqDomain_Calibration

mass_spectrum.molecular_search_settings.error_method = 'None'
mass_spectrum.molecular_search_settings.min_mz_error = -5
mass_spectrum.molecular_search_settings.max_mz_error = 5
mass_spectrum.molecular_search_settings.mz_error_range = 1

mass_spectrum.molecular_search_settings.usedAtoms['C'] = (1,90)
mass_spectrum.molecular_search_settings.usedAtoms['H'] = (4,200)
mass_spectrum.molecular_search_settings.usedAtoms['O'] = (0,20)
mass_spectrum.molecular_search_settings.usedAtoms['N'] = (0,0)
mass_spectrum.molecular_search_settings.usedAtoms['S'] = (0,0)


find_formula_thread = FindOxygenPeaks(mass_spectrum)
find_formula_thread.start()
find_formula_thread.join()

mspeaks_results = find_formula_thread.get_list_found_peaks()
    
calibrate = FreqDomain_Calibration(mass_spectrum, mspeaks_results)
calibrate.ledford_calibration()
#calibrate.step_fit()
mass_spectrum.clear_molecular_formulas()


### 5. Molecular formula search

In [ ]:
from corems.molecular_id.calc.ClusterFilter import ClusteringFilter
from corems.molecular_id.search.molecularFormulaSearch import SearchMolecularFormulas

#ClusteringFilter().filter_kendrick(mass_spectrum)

mass_spectrum.molecular_search_settings.min_mz_error = -1
mass_spectrum.molecular_search_settings.max_mz_error = 1
mass_spectrum.molecular_search_settings.mz_error_range = 1
mass_spectrum.molecular_search_settings.mz_error_average = 0
mass_spectrum.molecular_search_settings.min_abun_error = -30 # percentage 
mass_spectrum.molecular_search_settings.max_abun_error = 70 # percentage 

mass_spectrum.molecular_search_settings.isProtonated = True 
mass_spectrum.molecular_search_settings.isRadical= True 
mass_spectrum.molecular_search_settings.isadduct = True 

mass_spectrum.molecular_search_settings.usedAtoms['C'] = (1,90)
mass_spectrum.molecular_search_settings.usedAtoms['H'] = (4,200)
mass_spectrum.molecular_search_settings.usedAtoms['O'] = (1,20)
mass_spectrum.molecular_search_settings.usedAtoms['N'] = (0,0)
mass_spectrum.molecular_search_settings.usedAtoms['S'] = (0,0)

SearchMolecularFormulas(first_hit=True).run_worker_mass_spectrum(mass_spectrum)
print('ok')


### 6. Accessing the molecular formula  objects

In [ ]:
for mspeaks in mass_spectrum.sort_by_abundance():
   if mspeak.is_assigned: #or just if mspeak:
        for mf in mspeaks:
            print(mf.mz_theor,mf.dbe, mf.class_label, mf.to_string_formated)
   

In [ ]:
print (mass_spectrum.analyzer)

### 7. Export the results

In [ ]:
from corems.mass_spectrum.output.export import MassSpecExport

out_filename = 'neg_esi_srfa_1ppm_test'

exportMS= MassSpecExport(out_filename, mass_spectrum)
exportMS.run()

exportMS.output_type = 'csv'
exportMS.run()

exportMS.output_type = 'pandas'
exportMS.run()

# carefull if the file exits it will write another dataset to the scan number group 0
exportMS.output_type = 'hdf5'
exportMS.run()

df = exportMS.get_pandas_df()

print( df.head())